In [4]:
import pandas as pd
import numpy as np
import datetime
from collections import Counter

dataset = pd.read_csv("clean.csv")

In [13]:
dataset.head()

,Unnamed: 0,review_id,user_id,business_id,review_stars,review_date,user_review_count,user_average_stars,city,state,restaurant_stars,restaurant_review_count,category
0,0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,6,4.67,Montréal,QC,4.0,84,Diners
1,1,vm1b1keOzwHjtGZEPPuYXA,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5,2016-02-22,177,3.41,Montréal,QC,4.0,84,Diners
2,2,jUzausdZ_ujqe_n8BlBj-g,DVOOF0Z627DyrZ4XKQbTgA,AEx2SYEUJmTxVVB18LlCwA,5,2017-08-08,40,3.98,Montréal,QC,4.0,84,Diners
3,3,SXwA9KZ-Nc_hMARk_3cJ7g,5Ymfsf9fAYz-Ds_p0xawVQ,AEx2SYEUJmTxVVB18LlCwA,5,2013-03-29,79,4.52,Montréal,QC,4.0,84,Diners
4,4,oCRDwF3tszAkeszSfxwthg,5JoKz3mU42Cp906KRXDwJw,AEx2SYEUJmTxVVB18LlCwA,4,2009-01-17,3,4.00,Montréal,QC,4.0,84,Diners


In [5]:
nans = pd.Series(np.zeros(368),dtype=object)
rest = dataset[['review_date', 'state', 'category']]
rest.head()

,review_date,state,category
0,2016-05-28,QC,Diners
1,2016-02-22,QC,Diners
2,2017-08-08,QC,Diners
3,2013-03-29,QC,Diners
4,2009-01-17,QC,Diners


In [71]:

#State and category
print(rest.shape)
weekdays = rest[['state','category']].drop_duplicates()
weekdays['state_cat_revs'] = np.nan
print(weekdays.head())
vc = rest['state'].unique()
#vc = ['QC']
for num,i in enumerate(vc):
    cuisines = rest.loc[rest['state'] == i]['category'].unique()
    print(i)
    for j in cuisines:
        print(j)
        date = rest.loc[(rest['state'] == i) & (rest['category'] == j)]
        dates = []
        for dat in date['review_date']:
            year, month, day = (int(x) for x in dat.split('-'))    
            dates.append(datetime.date(year, month, day).weekday())
            
        wd_dict = dict(Counter(dates))
        #print(wd_dict)
        wd_counts = [wd_dict[k] for k in sorted( wd_dict.keys())]
        wd_counts = pd.Series([wd_counts], dtype='object')
        print(wd_counts)
        print(weekdays.loc[(weekdays['state'] == i) & (weekdays['category'] == j)])
        weekdays.loc[(weekdays['state'] == i) & (weekdays['category'] == j),'state_cat_revs'] = wd_counts


(2117138, 3)
    state        category  state_cat_revs
0      QC          Diners             NaN
84     QC  Canadian (New)             NaN
134    QC       Fast Food             NaN
195    QC        Bakeries             NaN
592    ON            Thai             NaN
QC
Diners
0    [379, 365, 338, 303, 312, 347, 399]
dtype: object
  state category  state_cat_revs
0    QC   Diners             NaN
Canadian (New)
0    [969, 800, 737, 781, 746, 889, 1046]
dtype: object
   state        category state_cat_revs
84    QC  Canadian (New)            NaN
Fast Food
0    [122, 113, 123, 117, 103, 107, 111]
dtype: object
    state   category state_cat_revs
134    QC  Fast Food            NaN
Bakeries
0    [202, 186, 167, 164, 163, 147, 186]
dtype: object
    state  category state_cat_revs
195    QC  Bakeries            NaN
Breakfast & Brunch
0    [868, 722, 688, 605, 658, 791, 1061]
dtype: object
     state            category state_cat_revs
2429    QC  Breakfast & Brunch            NaN
Vietnamese
0   

KeyboardInterrupt: 

In [3]:
#### WEEKDAYS ########

weekdays = rest[['state','category']].drop_duplicates()
#weekdays['state_cat_revs'] = np.nan
print(weekdays.head())
newdays = []
for stat_cat in weekdays.values:
    #print(stat_cat.shape)
    date = rest.loc[(rest['state'] == stat_cat[0]) & (rest['category'] == stat_cat[1])]
    dates = []
    for dat in date['review_date']:
        year, month, day = (int(x) for x in dat.split('-'))    
        dates.append(datetime.date(year, month, day).weekday())

    wd_dict = dict(Counter(dates))
    #print(wd_dict)
    wd_counts = [wd_dict[k] for k in sorted( wd_dict.keys())]
    wd_counts = pd.Series([wd_counts], dtype='object')
    #print(wd_counts)
    newdays.append(np.append(stat_cat,wd_counts))
    #print(newdays)

    state        category
0      QC          Diners
84     QC  Canadian (New)
134    QC       Fast Food
195    QC        Bakeries
592    ON            Thai


KeyboardInterrupt: 

In [83]:
pd.DataFrame(newdays).to_csv('weekdays.csv')

In [9]:
#### DAYS OF YEAR ######

weekdays = rest[['state','category']].drop_duplicates()
print(weekdays.head())
newdays = []
for stat_cat in weekdays.values:
    #print(stat_cat.shape)
    date = rest.loc[(rest['state'] == stat_cat[0]) & (rest['category'] == stat_cat[1])]
    dates = [dat[5:] for dat in date['review_date']]
    dates_counts = Counter(dates)
    del dates_counts['02-29']
    year = np.zeros(366,dtype=int)

    for date in dates_counts:
        dt = datetime.datetime(year=2019,month=int(date[:2]), day=int(date[3:]))
        #print(dt.timetuple().tm_yday)
        year[dt.timetuple().tm_yday] = dates_counts[date]
    year = list(year[1:])
    #print(wd_dict)
    wd_counts = pd.Series([year], dtype='object')
    #print(wd_counts)
    newdays.append(np.append(stat_cat,wd_counts))
    #print(newdays)

    state        category
0      QC          Diners
84     QC  Canadian (New)
134    QC       Fast Food
195    QC        Bakeries
592    ON            Thai


In [10]:
pd.DataFrame(newdays).to_csv('dayofyear.csv')